In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Loading
load the targeted data ( Cardiovascular Disease dataset: The dataset consists of 70 000 records of patients data, 11 features + target. [cardiovascular-disease-dataset](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) )

In [ ]:
# import all needed libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
import math

# load data as a dataframe with pandas 
sns.set();
df = pd.read_csv('../input/cardiovascular-disease-dataset/cardio_train.csv' , sep=';' , index_col = 0)
# sep = ';' is used for seperate the values from each other and buildthis data frame
df.head()

# Data Wrangling
to see if there is need to clean the data 

In [ ]:
# see the data information - columns , counts , types - 
df.info()

**first comment** we see that the age is represented in integers as we need but it represents the age with days and in this problem; it is not convient for the algorithm to deal with data in this represetnation.
so our first wrangling step is to convert the represetantion of age from days to years and approximate the age to an integer number.

In [ ]:
df.age = df.age / (365)
df.age = df.age.astype(np.int64)
df.age.head()

**second comment** that we will found that also the weight represented in float number but no need to represent it in floats; integer numbers will be suitable.
so we will convert the representation of the weight from float to integers

In [ ]:
df.weight = df.weight.astype(np.int64)
df.weight.describe()

due to this [article](http://answersafrica.com/skinniest-people.html#:~:text=Cathie%20Jung%2C%20born%20in%201937,inch%20waist%20(38.1%20cm).) we found the most skinnest adult on earth weigh around 30 kg so all of other data could be outliers so i prefer to neglect them

In [ ]:
# we can change our data frame to the df.weight > 35
df = df[df.weight > 35]
df.info()

In [ ]:
df.ap_hi.describe()

from this [article](https://www.health.harvard.edu/heart-health/reading-the-new-blood-pressure-guidelines) we found that the normal range of the systolic blood pressure range could be from 70:200, and from this draw we see that there is an outlire we should avoid it... 

and due to this photo too we could know the exact range of the systolic and dyastolic blood pressure

![](https://lh3.googleusercontent.com/proxy/BUMTJb6KWjNj4oEioNoXu2_i_kE8nadYQXqv315ZDMxnSDEOeepzy-3hQ8A7eNlaEJYBN3LkupVNsRRVHNGz_4hV0EEzbDSjlMKaJQkPfeqDxwAM1WgYUonLTM-g)

In [ ]:
df = df[(df.ap_hi < 200) & (df.ap_hi > 70 )]

In [ ]:
plt.plot(df.ap_lo)

from the previous we will find that the dyastolic pressure is way more than the average range 40:100 , so will try to see the description of this column**

In [ ]:
df.ap_lo.describe()

In [ ]:
df = df[(df.ap_lo < 100) &( df.ap_lo > 40 )]

In [ ]:
df.ap_lo.describe()

In [ ]:
df.info()

# Data Exploring
visualizing the data to see if there is any correlation or good insights to solve this problem


best way to start is to see the relations and the correlation between differenet columns and see if it would be anyway useuful to us

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(df.corr() , cmap = 'Greys' , annot = True)

from the correlation we cant find any important insights except that the api_hi (sytolic blood pressure) and in the api_lo (dyastolic blood pressure) is correlated with each other with 0.65 positive correlation.

In [ ]:
y = df.cardio
X = df.drop(['cardio'] , axis =1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

dec = tree.DecisionTreeClassifier()
log = LogisticRegression(random_state=0)
clf = RandomForestClassifier(max_depth=2, random_state=0)

models = {"Decision Tree" : dec ,
          "Random Forest" : clf,
         " Logistics Regression" : log
         }

scores = {}

for key, value in models.items():
    model = value
    model.fit(X_train, y_train)
    scores[key] = model.score(X_test, y_test)

scores_frame = pd.DataFrame(scores, index = ['Accuracy Score']).T
scores_frame.sort_values (by='Accuracy Score' , axis = 0 , ascending = False , inplace = True)
scores_frame